In [ ]:
import torchvision
import torchvision.transforms as transforms

from google.colab import drive
drive.mount('/content/drive')
save_path = '/content/drive/My Drive/Colab Notebooks/Save/'
import os
# 创建路径如果它不存在
os.makedirs(save_path, exist_ok=True)
previous_checkpoint = None  # 用来追踪上一个checkpoint的文件名


transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),  # 随机旋转
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # 颜色抖动
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train) #训练数据集
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)




for _, (x, y) in enumerate(trainset):
  print(x.shape, y)
  break





import torch


trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)   #生成一个个batch进行批训练，组成batch的时候顺序打乱取
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

for _, (x, y) in enumerate(trainloader):
  print(x.shape, y)
  break





import torch
import torch.nn as nn
import torch.nn.functional as F

torch.autograd.set_detect_anomaly(True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"你正在使用：{device}")

# 深度可分离卷积模块
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

# 残差块定义
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            DepthwiseSeparableConv(inchannel, outchannel, stride=stride),
            DepthwiseSeparableConv(outchannel, outchannel, stride=1)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        identity = x
        out = self.left(x)
        shortcut = self.shortcut(identity)
        out = out + shortcut  # Use out = out + shortcut instead of out += shortcut to ensure it's not inplace
        out = F.relu(out, inplace=False)
        return out


# ResNet主体架构
class ResNet(nn.Module):
    def __init__(self, block, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(block, 64,  1, stride=1)
        self.layer2 = self.make_layer(block, 128, 1, stride=2)
        self.layer3 = self.make_layer(block, 256, 1, stride=2)
        self.layer4 = self.make_layer(block, 512, 1, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# 实例化模型并移至设备
def ResNet18():
    return ResNet(ResidualBlock, num_classes=10)

net = ResNet18().to(device)
print(net)





def calc_param(model: nn.Module) -> int:
    params = list(model.parameters())
    param_size = 0
    for _param in params:
        _param_size = 1
        for _ in _param.size():
            _param_size *= _
        param_size += _param_size
    return param_size

print(f"The number of model parameters is:{calc_param(net)}")





import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=5e-4)

# 定义学习率调度器
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)



def load_checkpoint(model, optimizer, scheduler, filename='model_checkpoint.pth'):
    # 注意: 输入模型 & 优化器需要预先定义
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])

    # 也可以加载其他数据，例如 epoch 数或损失
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return model, optimizer, scheduler, epoch, loss

# 例子：加载状态
checkpoint_path = os.path.join(save_path, f'model_checkpoint_33.pth')
model, optimizer, scheduler, start_epoch, val_loss = load_checkpoint(net, optimizer, scheduler, checkpoint_path)






def compute_validation_loss(model, val_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_count = 0

    with torch.no_grad():  # No gradients needed
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item() * x.size(0)  # Multiply loss by batch size
            total_count += x.size(0)

    average_loss = total_loss / total_count
    return average_loss

for epoch in range(1):
    net.train()
    running_loss = 0.0
    for batch_idx, (x, y) in enumerate(trainloader, 0):
        inputs, labels = x.to(device), y.to(device)
        # 参数梯度置 0
        optimizer.zero_grad()
        # 前向推理
        outputs = net(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        # 后向传递梯度计算
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (batch_idx + 1) % 100 == 0:
            print(f"[EPOCH {epoch + 1:03} | IT {batch_idx + 1:05}] LOSS: {running_loss / batch_idx: .3f}")
    # 假设val_loss是你在验证集上计算得到的损失
    val_loss = compute_validation_loss(net, trainloader, device)
    scheduler.step(val_loss)


    # 保存模型权重、优化器和调度器状态
    # 删除上一个checkpoint
    if previous_checkpoint is not None:
        try:
            os.remove(previous_checkpoint)
            print(f"Removed previous checkpoint: {previous_checkpoint}")
        except OSError as e:
            print(f"Error: {previous_checkpoint} : {e.strerror}")

    checkpoint = {
        'epoch': epoch + 1,
        'state_dict': net.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict(),
        'loss': val_loss,
    }
    checkpoint_path = os.path.join(save_path, f'model_checkpoint_{epoch+1}.pth')
    torch.save(checkpoint, checkpoint_path)
    # 更新previous_checkpoint变量
    previous_checkpoint = checkpoint_path
    print(f"Checkpoint saved at {checkpoint_path}")





def calc_acc(model: nn.Module, testloader: torch.utils.data.DataLoader) -> float:
  net.eval()
  total = 0
  correct = 0
  with torch.no_grad():
    for _, (x, y) in enumerate(testloader):
      x, y = x.to(device), y.to(device)
      outputs = model(x)
      _, predicted = torch.max(outputs.data, 1)
      total += y.size(0)
      correct += (predicted == y).sum().item()

  return correct / total


print(f"""
- 模型参数量：{calc_param(net)}
- 模型测试准确率：{calc_acc(net, testloader)}""")

## 课程任务

---

你需要自己从头开始设计和创建一个分类器，在 CIFAR 10 数据上进行训练和测试。

要求：
- 模型参数量不超过 `ResNet18`，即 11689512 ，（通过 `calc_param(torchvision.models.resnet18())`计算得到）。
- 模型达到尽可能高的准确率。